In [1]:
# default_exp modules

This contains pytorch modules which allow for optimization of intrinsic/extrinsic parameters

# Import

In [2]:
# export
import torch
import torch.nn as nn

from camera_calib.utils import *

In [3]:
from IPython.core.debugger import set_trace

# Utility

In [4]:
# export
def tensors2parameters(args): return args_loop(args, lambda arg: nn.Parameter(arg))

# Inversible Modules

Some things:
* I want to be able to have a `Module`, call `Inverse()` on it, and have the `Module` and it's `Inverse` share the same parameters, yet the inverse does the inverse of the module
* I want to ensure some intuitive things, like: `Inverse(Inverse(Module)) == Module`
* It's useful to be able to access some parameters in an "inverse-aware" reconstructed form, so I've implemented `get_param()`. i.e. if a `Rotation` is `Inverse`d and I want to get the rotation matrix, calling the `get_param()` method should return the transposed version

For inversible modules, inherit this:

In [5]:
# export
class Inversible(nn.Module):
    def __init__(self):
        super().__init__()
       
    def forward_param(self): raise NotImplementedError('Please implement forward_param() method')
    def inverse_param(self): raise NotImplementedError('Please implement inverse_param() method')
        
    # Call this method to get a copy of "inverse aware" reconstructed parameters
    def get_param(self):
        with torch.no_grad():
            return self.forward_param().detach().clone() 
    
    def inverse(self): raise NotImplementedError('Please implement inverse() method')

Do `Inverse(Module)` to take its inverse:

In [6]:
# export
class Inverse(Inversible): # Note that Inverse is itself Inversible...
    def __init__(self, m):
        super().__init__()
        self.m = m
        
    def __repr__(self):
        return f'{self.__class__.__name__}({self.m.__repr__()})'
    
    def forward_param(self): return self.m.inverse_param()
    def inverse_param(self): return self.m.forward_param()
    
    def forward(self, x): return self.m.inverse(x)
    def inverse(self, x): return self.m.forward(x)

Have a basic test to test inversibility

In [7]:
# export
def assert_inversible(m, x, y, **kwargs):
    assert_allclose(m(x),          y, **kwargs)
    assert_allclose(Inverse(m)(y), x, **kwargs)

In [8]:
a = Inversible(); b = Inverse(a); c = Inverse(b)
a,b,c

(Inversible(), Inverse(Inversible()), Inverse(Inverse(Inversible())))

# Translation module

This performs a simple 3D translation; the inverse is simply subtracting the translation

In [9]:
# export
class Translation(Inversible):
    def __init__(self, t):
        super().__init__()
        self.t = tensors2parameters(t)
        
    def __repr__(self):
        t_x, t_y, t_z = self.t
        return f'{self.__class__.__name__}(tx:{t_x:.4} ty:{t_y:.4} tz:{t_z:.4})'
        
    def forward_param(self): return  self.t
    def inverse_param(self): return -self.forward_param()
        
    def forward(self, ps, inverse=False): 
        t = self.forward_param() if not inverse else self.inverse_param()
        return ps + t
    def inverse(self, ps): return self.forward(ps, inverse=True)

In [10]:
t = torch.FloatTensor([1,2,3])
a = Translation(t); b = Inverse(a);
a, b

(Translation(tx:1.0 ty:2.0 tz:3.0), Inverse(Translation(tx:1.0 ty:2.0 tz:3.0)))

In [11]:
ps = torch.rand(4,3)
ps

tensor([[0.4434, 0.7333, 0.4652],
        [0.6086, 0.4025, 0.1358],
        [0.1858, 0.7119, 0.9138],
        [0.5036, 0.6143, 0.9526]])

In [12]:
assert_inversible(a, ps, ps+t, atol=1e-4)
assert_inversible(b, ps, ps-t, atol=1e-4)

In [13]:
assert_allclose(a.get_param(),  t)
assert_allclose(b.get_param(), -t)

# Rotation module

This performs a simple 3D rotation; the inverse is simply the transpose. Implement the reconstruction in the `r2R()` method for child classes.

In [14]:
# export
class Rotation(Inversible):
    def __init__(self):
        super().__init__()
        
    def r2R(self):
        raise NotImplementedError('Please implement rotation matrix reconstruction')       
    
    def forward_param(self): return self.r2R()
    def inverse_param(self): return self.forward_param().T
        
    def forward(self, ps, inverse=False): 
        R = self.forward_param() if not inverse else self.inverse_param()
        return pmm(ps, R)
    def inverse(self, ps): return self.forward(ps, inverse=True)

### Euler

In [15]:
# export
class EulerRotation(Rotation):
    def __init__(self, R):
        super().__init__()
        self.euler = tensors2parameters(R2euler(R))
    
    def __repr__(self):
        e_x, e_y, e_z = self.euler
        return f'{self.__class__.__name__}(ex:{e_x:.4} ey:{e_y:.4} ez:{e_z:.4})'
    
    def r2R(self): return euler2R(self.euler)

In [16]:
R = euler2R(torch.FloatTensor([1,2,3]))
a = EulerRotation(R); b = Inverse(a)
a,b

(EulerRotation(ex:-2.142 ey:1.142 ez:-0.1416),
 Inverse(EulerRotation(ex:-2.142 ey:1.142 ez:-0.1416)))

In [17]:
ps = torch.rand(4,3)
ps

tensor([[0.0563, 0.0262, 0.7179],
        [0.2494, 0.3563, 0.0611],
        [0.1485, 0.0566, 0.3633],
        [0.4392, 0.1696, 0.8012]])

In [18]:
assert_inversible(a, ps, pmm(ps, R  ), atol=1e-4)
assert_inversible(b, ps, pmm(ps, R.T), atol=1e-4)

In [19]:
assert_allclose(a.get_param(), R)
assert_allclose(b.get_param(), R.T)

# InversibleSequential Module

Having inversible composible modules will be super useful. This is basically nn.Sequential but I've made it inversible. Note that:
`y = f(g(h(x)))` implies that `x = h^-1(g^-1(f^-1(y)))`. So basically just reverse the order and take the inverse of each function to invert the whole thing.

In [20]:
# export
class InversibleSequential(Inversible):
    def __init__(self, ms):
        super().__init__()
        self.ms = nn.ModuleList(ms)  
    
    def forward(self, x):
        for m in self.ms: 
            x = m.forward(x)
        return x
    
    def inverse(self, x):
        for m in reversed(self.ms):
            x = m.inverse(x)
        return x

# Rigid

Rigid transform is a rotation followed by translation.

In [21]:
# export
class Rigid(InversibleSequential):
    def __init__(self, R, t, Rotation=EulerRotation):
        super().__init__([Rotation(R), Translation(t)]) # NOTE: rotation happens first
        
    def forward_param(self):
        return Rt2M(self.ms[0].forward_param(), # R 
                    self.ms[1].forward_param()) # t
        
    def inverse_param(self): return invert_rigid(self.forward_param())

In [22]:
R, t = euler2R(tuple(torch.FloatTensor([1,2,3]))), torch.FloatTensor([1,2,3])
a = Rigid(R, t); b = Inverse(a)
a,b

(Rigid(
   (ms): ModuleList(
     (0): EulerRotation(ex:-2.142 ey:1.142 ez:-0.1416)
     (1): Translation(tx:1.0 ty:2.0 tz:3.0)
   )
 ), Inverse(Rigid(
   (ms): ModuleList(
     (0): EulerRotation(ex:-2.142 ey:1.142 ez:-0.1416)
     (1): Translation(tx:1.0 ty:2.0 tz:3.0)
   )
 )))

In [23]:
ps = torch.rand(4,3)
ps

tensor([[0.6007, 0.9804, 0.0665],
        [0.1926, 0.0107, 0.3777],
        [0.5983, 0.9417, 0.4780],
        [0.9855, 0.6837, 0.7992]])

In [24]:
M = Rt2M(R,t)
M_inv = invert_rigid(M)
R_inv, t_inv = M2Rt(M_inv)

In [25]:
assert_inversible(a, ps, pmm(ps, R    ) + t,     atol=1e-4)
assert_inversible(b, ps, pmm(ps, R_inv) + t_inv, atol=1e-4)

In [26]:
assert_allclose(a.get_param(), M)
assert_allclose(b.get_param(), M_inv)

Create a wrapper for composing multiple `Rigid`s

In [27]:
# export
class Rigids(InversibleSequential):
    def __init__(self, rigids):
        super().__init__(rigids)
    
    def forward_param(self):
        M = self.ms[0].forward_param()
        for m in self.ms[1:]:
            M = mult_rigid(m.forward_param(), M)
        return M
    
    def inverse_param(self): return invert_rigid(self.forward_param())

In [28]:
R_a, t_a = euler2R(torch.FloatTensor([ 1, 2, 3])), torch.FloatTensor([1,2,3])
a = Rigid(R_a, t_a)
R_b, t_b = euler2R(torch.FloatTensor([.5, 1, 1])), torch.FloatTensor([3,2,2])
b = Inverse(Rigid(R_b, t_b))
R_c, t_c = euler2R(torch.FloatTensor([.3,.1, 0])), torch.FloatTensor([3,3,3])
c = Rigid(R_c, t_c)

In [29]:
d = Rigids([a,b,c])

In [30]:
M_a = Rt2M(R_a, t_a)
M_b = Rt2M(R_b.T, -R_b.T@t_b)
M_c = Rt2M(R_c, t_c)
M_d = M_c@M_b@M_a

In [31]:
R_d, t_d = M2Rt(M_d)

In [32]:
assert_inversible(d, ps, pmm(ps, R_d) + t_d, atol=1e-4)

In [33]:
assert_allclose(d.get_param(), M_d)

# Normalize

This will normalize 3D coordinates (i.e. project points to "unit" image plane).

In [34]:
# export
class Normalize(nn.Module): # Note: Not Inversible
    def __init__(self):
        super().__init__()
        
    def forward(self, ps): return normalize(ps)

In [35]:
ps = torch.rand(4,3)
ps

tensor([[0.5071, 0.3357, 0.1459],
        [0.6645, 0.3533, 0.1953],
        [0.4507, 0.6132, 0.2910],
        [0.3871, 0.3325, 0.6962]])

In [36]:
assert_allclose(Normalize()(ps), ps[:,0:2]/ps[:,[2]])

# Augment

Adds `one`s to input array; note this is `Inversible`

In [37]:
# export
class Augment(Inversible):
    def __init__(self):
        super().__init__()

    def forward(self, ps): return augment(ps)
    def inverse(self, ps): return deaugment(ps)

Test to make sure `one`s are added and removed

In [38]:
ps = torch.rand(4,2)
ps

tensor([[0.1338, 0.8614],
        [0.6655, 0.4272],
        [0.9090, 0.8333],
        [0.9843, 0.5582]])

In [39]:
assert_inversible(Augment(), ps, torch.cat([ps, ps.new_ones(len(ps),1)], dim=1))

# Distortion

This will apply distortion model to 2D points. Note for now I didn't find much commonality so I didn't make a base class. Also note that distortions are typically not inversible (at least not analytically to my knowledge), so exclude for now.

### No distortion

In [40]:
# export
NoDistortion = nn.Identity

### Heikkila97 distortion

In [41]:
# export
class Heikkila97Distortion(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.d = tensors2parameters(d)
    
    def __repr__(self):
        k1, k2, p1, p2 = self.d
        return f'{self.__class__.__name__}(k1:{k1:.4} k2:{k2:.4} p1:{p1:.4} p2:{p2:.4})'
    
    def forward(self, ps):
        k1, k2, p1, p2 = self.d
        xs, ys = ps[:,0], ps[:,1]
        
        # Radial distortion
        rs = xs**2 + ys**2
        xs_r = xs*(1 + k1*rs + k2*rs**2)
        ys_r = ys*(1 + k1*rs + k2*rs**2)

        # Decentering distortion 
        xs_d = xs_r + 2*p1*xs*ys + p2*(3*xs**2 + ys**2)
        ys_d = ys_r + p1*(xs**2 + 3*ys**2) + 2*p2*xs*ys
        
        return stackify((xs_d, ys_d), dim=1)

In [42]:
distortion = Heikkila97Distortion(torch.FloatTensor([.01, 0.02, 0.03, 0.04]))
distortion

Heikkila97Distortion(k1:0.01 k2:0.02 p1:0.03 p2:0.04)

In [43]:
ps = torch.FloatTensor([[0.1900, 0.2467],
                        [0.9817, 0.7349],
                        [0.2432, 1.6161],
                        [0.2194, 0.4353]])
assert_allclose(distortion(ps), torch.FloatTensor([[0.1998, 0.2573],
                                                   [1.2215, 0.9145],
                                                   [0.4196, 2.1581],
                                                   [0.2393, 0.4630]]), atol=1e-4)

### Wang08 distortion

In [44]:
# export
class Wang08Distortion(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.d = tensors2parameters(d)
    
    def __repr__(self):
        k1, k2, p, t = self.d
        return f'{self.__class__.__name__}(k1:{k1:.4} k2:{k2:.4} p:{p:.4} t:{t:.4})'
    
    def forward(self, ps):
        k1, k2, p, t = self.d
        xs, ys = ps[:,0], ps[:,1]
        
        # Radial distortion
        rs = xs**2 + ys**2
        xs_r = xs*(1 + k1*rs + k2*rs**2)
        ys_r = ys*(1 + k1*rs + k2*rs**2)

        # Image plane (small angle approximation) rotation distortion
        xs_d = xs_r/(-p*xs_r + t*ys_r + 1)
        ys_d = ys_r/(-p*xs_r + t*ys_r + 1)
        
        return stackify((xs_d, ys_d), dim=1)

NOTE: possibly refactor the radial distortion part since it appears twice

In [45]:
distortion = Wang08Distortion(torch.FloatTensor([.01, 0.02, 0.03, 0.04]))
distortion

Wang08Distortion(k1:0.01 k2:0.02 p:0.03 t:0.04)

In [46]:
ps = torch.FloatTensor([[0.1900, 0.2467],
                        [0.9817, 0.7349],
                        [0.2432, 1.6161],
                        [0.2194, 0.4353]])
assert_allclose(distortion(ps), torch.FloatTensor([[0.1894, 0.2460],
                                                   [1.0409, 0.7792],
                                                   [0.2665, 1.7711],
                                                   [0.2178, 0.4321]]), atol=1e-4)

# Camera matrix

This performs the application of the Camera matrix. Implement the reconstruction in the `a2A()` method for child classes.

In [47]:
# export
def a2A(alpha_x, alpha_y, x_o, y_o):
    zero, one = alpha_x.new_tensor(0), alpha_x.new_tensor(1)

    return stackify(((alpha_x,    zero, x_o),
                     (   zero, alpha_y, y_o),
                     (   zero,    zero, one)))

Test to make sure reconstruction is correct

In [48]:
a = torch.FloatTensor([1.5, -2.5, 3.5])
A = a2A(alpha_x=a[0], alpha_y=a[0], x_o=a[1], y_o=a[2])
assert_allclose(A, torch.FloatTensor([[ 1.5,  0.0, -2.5],
                                      [ 0.0,  1.5,  3.5],
                                      [ 0.0,  0.0,  1.0]]))

In [49]:
# export
class Cam(Inversible):
    def __init__(self):
        super().__init__()
    
    def a2A(self):
        raise NotImplementedError('Please implement camera matrix reconstruction')
    
    def forward_param(self): return self.a2A()
    def inverse_param(self): return torch.inverse(self.forward_param()) # Overwrite this in child classes
        
    def forward(self, ps, inverse=False):
        A = self.forward_param() if not inverse else self.inverse_param()
        return pmm(ps, A, aug=True)
    def inverse(self, ps): return self.forward(ps, inverse=True)

### Single focal length

Assumes there is only a single `alpha`

In [50]:
# export
class CamSF(Cam):
    def __init__(self, A):
        super().__init__()
        assert_allclose(A[0,0], A[1,1]) # Check that alpha_x and alpha_y are the same
        self.a = tensors2parameters(A[[0,0,1],[0,2,2]])
    
    def __repr__(self):
        alpha, x_o, y_o = self.a
        return f'{self.__class__.__name__}(alpha:{alpha:.4} x_o:{x_o:.4} y_o:{y_o:.4})'
    
    def a2A(self):
        alpha, x_o, y_o = self.a
        return a2A(alpha_x=alpha, alpha_y=alpha, x_o=x_o, y_o=y_o)
    
    def inverse_param(self): # Overwrite parent method
        alpha, x_o, y_o = self.a
        return a2A(alpha_x=1/alpha, 
                   alpha_y=1/alpha, 
                   x_o=-x_o/alpha, 
                   y_o=-y_o/alpha)

In [51]:
a = torch.FloatTensor([1.5, -2.5, 3.5])
A = a2A(alpha_x=a[0], alpha_y=a[0], x_o=a[1], y_o=a[2])
cam = CamSF(A)
cam

CamSF(alpha:1.5 x_o:-2.5 y_o:3.5)

Test

In [52]:
ps = torch.rand(4,2)
ps

tensor([[0.2899, 0.1666],
        [0.8523, 0.8353],
        [0.1051, 0.0022],
        [0.7013, 0.2706]])

In [53]:
assert_inversible(cam, ps, pmm(ps, A, aug=True), atol=1e-4)

In [54]:
assert_allclose(cam.get_param(), A)

# Build

In [55]:
build_notebook()

<IPython.core.display.Javascript object>

Converted modules.ipynb.
